In [5]:

def decor(fn):
    return 2

@decor #等价于 add = decor(add),在被加载是就已经运行装饰器替换了原函数，且只运行一次
def add():
    return 1
print(add)



2


In [1]:
def decor(fn):
    def wrapper():
        """wo shi decorator"""
        return 2
    return wrapper

@decor #等价于 add = decor(add),在被加载是就已经运行装饰器替换了原函数，且只运行一次
def add():
    return 1

#此时name为wrapper，即此时的调用已经是在调用docor里的wrapper了，说明在加载时函数add已经被装饰器替换成了新函数
print(add.__name__)
print(add.__doc__) #此时的doc都被替换成了 decor的了
print(add())




wrapper
wo shi decorator
2


In [13]:
# 被装饰函数带参数
def decor(fn):
#     根据需求而定参数
    def wrapper(*args, **kwargs):
        fn()
        return args
    return wrapper

@decor #等价于 add = decor(add)
def add(x, y, z):
    return 1

#此时name为wrapper，即此时的调用已经是在调用docor里的wrapper了，说明在加载时函数add已经被装饰器替换成了新函数
print(add.__name__)
rint(add.__doc__) #此时的doc都被替换成了 decor的了
print(add(1,2,3))

# 其实这边add(1,2,3)其实调用的是 decor下面的wrapper(1,2,3), 因为此时的add 就是函数 wrapper
# 其实调用顺序是： decor(add)(1,2,3)

wrapper
(1, 2, 3)


In [16]:
# 装饰函数带参数
# 由上面的可以理解@decor 的作用等价于 add = docor(add)
# @后面的 docor 是函数的引用, @语法糖会自己把被装饰的函数作为参数调用@后面的函数即 add = doctor(add)
# 那么@后面只要是函数的引用就行了对吧，那是不是@后面可以跟一个函数调用，此调用返回一个函数的引用，即上面的decor函数

# outdecor函数调用后返回decor的函数引用
def outdecor(a, b):
    def decor(fn):
    #     根据需求而定参数
        def wrapper(*args, **kwargs):
            fn(*args)
            return args
        return wrapper
    return decor

@outdecor(1,2) #等价于 add = outdecor(1,2)(add)
def add(x, y, z):
    return 1

print(add.__name__)
print(add(1,2,3))

#上面的@outdecor(1,2),真实操作就是，先调用了outdecor(1,2) 返回了一个内层的函数decor的引用，这样就把装饰器的参数传进去了(由闭包实现)


wrapper
(1, 2, 3)


In [32]:
# 由开发需要我们需要把add的__doc__改回自己原本的,或者还原其他属性

# 我们也可以写一个装饰器，还原属性
def restore_property(fn): #这边的fn就是接受的add函数的
    def wrapper(wrappered):# 这边的wrappered是接受 outdecor中的wrapper的， 
        #这里面没有对wrappered函数进行调用，只是改动了wrappered对属性， 所以没有接受参数对那层函数
        wrappered.__name__ = fn.__name__
        wrappered.__doc__ = fn.__doc__
        #这个return一定要， 因为在下面的@restore_property(fn)需要有一个函数 func(wrapper)
        #并返回outdecor中的wrapper， 即上面讲的@后面必需是个函数对引用，或一个函数对调用且调用后返回一个函数对引用
        return wrappered
    return wrapper


def outdecor(a, b):
    def decor(fn):
        @restore_property(fn) #装饰器里的装饰器
        def wrapper(*args, **kwargs):
            """wo shi decorator"""
            enter = fn(*args)
            return enter #这个return 可有可无
        return wrapper #这边的wrapper 其实就是上面return 回来的wrappered
    return decor

@outdecor(1,2) #等价于 add = outdecor(1,2)(add)
def add(x, y, z):
    """wo shi add """
    return x + y + z

print(add.__name__)
print(add.__doc__)
print(add(1,2,3))



add
wo shi add 
6


In [40]:

# 用类写一个带参数的装饰器, 且还原doc等属性
class RestoreProperty(object):
    def __init__(self, fn):
        self.fn = fn

    def __call__(self, wrappered):
        wrappered.__name__ = self.fn.__name__
        
        wrappered.__doc__ = self.fn.__doc__
        return wrappered

    
class Decor(object):
    def __init__(self, c, d):
        self.c = c
        self.d = d
        
    def __call__(self, fn):
        @RestoreProperty(fn)
        def wrapper(x, y, z):
            """wo shi 类装饰器"""
            print('before')
            enter = fn(x, y, z)
            print('after')
            return enter
        return wrapper



@Decor(1,2) #等价于 add = outdecor(1,2)(add)
def add(x, y, z):
    """wo shi add """
    return x + y + z

print(add.__name__)
print(add.__doc__)
print(add(1,2,3))

add
wo shi add 
before
after
6
